In [399]:
import urllib2
import pandas as pd
import re
import operator
from datetime import datetime, date, time
from bs4 import BeautifulSoup
import numpy as np
import itertools
import math
import time
from trueskill import Rating, quality_1vs1, rate_1vs1, quality, rate, global_env
BETA=4.166666666666667
from IPython.core.display import display, HTML
import csv
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
## Configuration Settings
match_start_id = 1
match_end_id = 1900
player_start_id = 1
player_end_id = 190

# Scrape and Parse URL

In [3]:
## SOUP ATTRIBUTES
def make_soup(url, _id):
    quote_page = url+str(_id)  # specify the url
    page = urllib2.urlopen(quote_page)  # query the website and return the html to the variable ‘page’
    return BeautifulSoup(page, 'html.parser')

def get_header(_soup, _remove, _type):
    h = _soup.find('h1').text.strip().replace(_remove,'')
    if _type == 'event':
        event_date = datetime.datetime.strptime(h, '%A, %B %d, %Y')
        h = {'datetime': event_date.strftime('%Y%m%d'),'day': event_date.strftime('%A')}
        h = h['datetime']
    return h

def get_tag(_soup, _remove, _tag):
    t = _soup.find(_tag).text.strip().replace(_remove,'')
    return t

def get_table_data(_soup):
    name_box = _soup.find('table')
    name_rows = name_box.find_all('tr') 
    strip = [a.text.strip().replace('\r\n','') for a in name_rows]
    strip = [a.split('\n') for a in strip]
    return strip

def scrape_url(_url, _id):
    soup = make_soup(_url, _id)
    table = get_table_data(soup)
    return soup, table

# Teams

## Teams > Scrape Team General Team Info

In [4]:
def scrape_team_info(division='all'):
    url = 'http://collegebeachvb.com/colleges'
    id_ = ''
    table = get_table_data(make_soup(url, id_))

    my_labels = table[0]
    my_results = table[1:]
    df_ = pd.DataFrame.from_records(my_results,columns=my_labels) 

    if division != 'all':
        df_ = df_[df_.Division == division][['College', 'Team Name', 'Location', 'Division']]
    else:
        df_ = df_[['College', 'Team Name', 'Location', 'Division']]
    
    return df_

df_teams = scrape_team_info()
df_teams

,College,Team Name,Location,Division
0,Abilene Christian,Wildcats,"Abilene, Tx",DI
1,Antelope Valley College,Marauders,"Los Angeles, Ca",Community College
2,Arizona,Wildcats,"Tucson, Az",DI
3,ASU,Sun Devils,"Tempe, Az",DI
4,Austin Peay State,Governors,"Clarksville, Tn",DI
5,Bakersfield College,Renegades,"Bakersfield, Ca",Community College
6,Benedictine,Ravens,"Mesa, Az",NAIA
7,Boise State,Broncos,"Boise, Id",DI
8,Cabrillo,Seahawks,"Aptos, Ca",Community College
9,Cal,Bears,"Berkeley,Ca",DI


# Players
## Players > Scrape Player Info

In [13]:
# This is only for 2018 rosters

def scrape_player_info(start= player_start_id, end= player_end_id):
    url = 'http://collegebeachvb.com/players/collegeplayers/'
    d = {}
    temp = None
    for i in range(start, end):
        print i,
        soup, table = scrape_url(url, i)
        header = soup.find('h1').text.strip().replace('College Players for ','')
        if (len(table) > 1) :
            if (len(table[1:][1]) == 4): 
                my_results = table[1:]
                my_labels = table[0]
                col_numbers = len(table[1:][0])
                my_results = table[1:]

                d[i] = pd.DataFrame.from_records(my_results, columns=['Name', 'Year', 'Height', 'Hometown/HS'])
                d[i]['Team'] = header
                d[i]['Name'] = d[i]['Name'].apply(lambda x: x.strip())
                #print d
            elif (len(table[1:][1]) == 1): 
                d[i] = pd.DataFrame.from_records(my_results, columns=['Name', 'Year', 'Height', 'Hometown/HS'])
                d[i]['Team'] = header
                d[i]['Name'] = d[i]['Name'].apply(lambda x: x.strip())
            elif (len(table[1:][1]) == 2): 
                d[i] = pd.DataFrame.from_records(my_results, columns=['Name', 'Year', 'Height', 'Hometown/HS'])
                d[i]['Team'] = header
                d[i]['Name'] = d[i]['Name'].apply(lambda x: x.strip())
            else:
                print i, len(table[1:][1])
                d[i] = pd.DataFrame.from_records(my_results, columns=['Name', 'Year', 'Height', 'Hometown/HS'])
                d[i]['Team'] = header
                d[i]['Name'] = d[i]['Name'].apply(lambda x: x.strip())
            
        temp = pd.concat([d[k] for k in d.keys()], axis=0)
                
    return temp

df_players = scrape_player_info()
df_players

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189

,Name,Year,Height,Hometown/HS,Team
0,Alyssa Slagerman,Fr,5'7,"Brentwood, Ca",UCLA
1,Cami Sanchez,Fr,5'10,"Fountain Valley, Ca",UCLA
2,Chanti Holroyd,So,5-9,"Mountain View, California",UCLA
3,Elise Zappia,Sr,5'8,"Huntington Beach, California",UCLA
4,Hannah Phair,Fr,5'10,"Huntington Beach, Ca",UCLA
5,Izzy Carey,Jr,5-11,"Westlake Village, California",UCLA
6,Jamie McQuarrie,Fr,6'0,"Encinitas, Ca",UCLA
7,Lexi Jasper-Baylin,Sr,5'6,"El Cajon, Ca",UCLA
8,Lily Justine,So,5-10,"Chico, California",UCLA
9,Mac May,Fr,6'3,"Dubuque, IA",UCLA


In [406]:
df_players.to_csv("beachvb_players.csv", encoding='utf-8')
df_teams.to_csv("beachvb_teams.csv", encoding='utf-8')
df_matchups.to_csv("beachvb_matchups.csv", encoding='utf-8')
df_player_rating.to_csv("beachvb_playerrating.csv", encoding='utf-8')
df_2018_d1_matchups.to_csv("beachvb_2018d1_matchups.csv", encoding='utf-8')
df_2018_matchups.to_csv("beachvb_2018_matchups.csv", encoding='utf-8')
df_2018_d1p1_matchups.to_csv("beachvb_2018d1p1_matchups.csv", encoding='utf-8')
df_2018_d1p2_matchups.to_csv("beachvb_2018d1p2_matchups.csv", encoding='utf-8')
df_2018_d1p3_matchups.to_csv("beachvb_2018d1p3_matchups.csv", encoding='utf-8')
df_2018_d1p4_matchups.to_csv("beachvb_2018d1p4_matchups.csv", encoding='utf-8')
df_2018_d1p5_matchups.to_csv("beachvb_2018d1p5_matchups.csv", encoding='utf-8')

# Matchups
## Matchups > Scrape Matchup Info

In [117]:
def scrape_matchups(start=match_start_id, end=match_end_id):
    url = 'http://collegebeachvb.com/scores/competition/'
    d = {}
    h = {}
    for i in range(start,end):
        print i
        try:
            table = scrape_url(url, i)[1]
            soup = scrape_url(url, i)[0]

            header = soup.find('h1').text.strip().replace('Competition on ','')
            header = datetime.strptime(str(header), '%A, %B %d, %Y').strftime('%Y%m%d')
            
            d[i] = table
            h[i] = header
        except:
            print "need to wait...."
            time.sleep(360)
            table = scrape_url(url, i)[1]
            soup = scrape_url(url, i)[0]

            header = soup.find('h1').text.strip().replace('Competition on ','')
            header = datetime.strptime(str(header), '%A, %B %d, %Y').strftime('%Y%m%d')

            d[i] = table
            h[i] = header
    return d, h

#s =scrape_matchups(780, 1870)

780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023


# Matchups
## Matchups > Select Matchups

In [119]:
def select_matchups(startdate, enddate, df=df_matchups, division='DI'):
    temp = df[df['Event Date'].astype(int) >  startdate]
    temp = temp[temp['Event Date'].astype(int) < enddate]
    df_team_players = df_players.merge(df_teams, how='inner', left_on='Team', right_on='College')
    temp = temp.merge(df_team_players, how='left', left_on='TeamAPlayer1', right_on='Name')
    temp['TeamAPlayer1'] = temp['TeamAPlayer1'].apply(lambda x: x.strip())
    temp['TeamAPlayer2'] = temp['TeamAPlayer2'].apply(lambda x: x.strip())
    temp['TeamBPlayer1'] = temp['TeamBPlayer1'].apply(lambda x: x.strip())
    temp['TeamBPlayer2'] = temp['TeamBPlayer2'].apply(lambda x: x.strip())
    temp = temp[temp['Division'] == division].iloc[:,[0,1,2,3,4,5,6,7,8]].drop_duplicates()
    temp = temp.sort_values('Event Date', ascending=True)    
    return  temp

## Matchups > Prepare Matchup Data

In [121]:
def prepare_matchup_data(m_dict):
    s_new = s[0]
    d_new = s[1]
    m = {}

    temp = None
    for a in s_new:
        print a,
        for b in s_new[a]:
            if len(b) == 5:
                team1 =  b[1]
                team2 =  b[2]
                #print team1, team2, "\n"
            elif len(b) == 7:
                if team1 == (b[5].replace('(', '').replace(')', '')):
                    winner = team1
                    loser = team2
                else:
                    winner = team2
                    loser = team1

                m[a] = [b[0], b[1], b[2], b[3], b[4], winner, loser,a,  d_new[a]]
                #print a
                #print m
                df_ = pd.DataFrame.from_dict(m, orient='index')
                df_.columns=['Position','TeamAPlayer1', 'TeamAPlayer2', 'TeamBPlayer1', 'TeamBPlayer2',\
                                                                   'Winner', 'Loser', 'Compid', 'Event Date' ]

                temp = pd.concat([temp,df_], axis=0)
    return temp



df_matchups = prepare_matchup_data(s)
df_matchups.sort_values('Event Date', ascending=True)
#df_2018_matchups = select_matchups( 20180000, 20180900).sort_values('Event Date', ascending=True)
#df_2018_matchups.sort_values('Event Date', ascending=True).Compid.unique()

780 781 782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 881 882 883 884 885 886 887 888 889 890 891 892 893 894 895 896 897 898 899 900 901 902 903 904 905 906 907 908 909 910 911 912 913 914 915 916 917 918 919 920 921 922 923 924 925 926 927 928 929 930 931 932 933 934 935 936 937 938 939 940 941 942 943 944 945 946 947 948 949 950 951 952 953 954 955 956 957 958 959 960 961 962 963 964 965 966 967 968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020 1021 1022 1023 

,Position,TeamAPlayer1,TeamAPlayer2,TeamBPlayer1,TeamBPlayer2,Winner,Loser,Compid,Event Date
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223
788,5,Katie Schueman,Kaxandra Naranjo,Ivelisse Saint-Clair,Ashley Kube,Webber,Warner,788,20170223


In [409]:
df_matchups


,Position,TeamAPlayer1,TeamAPlayer2,TeamBPlayer1,TeamBPlayer2,Winner,Loser,Compid,Event Date
780,1,Alexandra Spadaro,Nikki McDaniel,Kristen Brooks,Samantha Perillo,Webber,Stevenson,780,20170316
780,2,Carlee Baxter,Tessa Colden,Lauren Weaver,Katie Leftridge,Webber,Stevenson,780,20170316
780,3,Brianna Baez,Michelle Calderon,Trang Christensen,Grace Coleman,Webber,Stevenson,780,20170316
780,4,Amber Lake,Hannah Tannone,Darian Hileman,Megan Hren,Webber,Stevenson,780,20170316
780,5,Katie Schueman,Laura Embry,Deanna Bradbury,Chelsea Ireland,Webber,Stevenson,780,20170316
781,1,Lauren Weaver,Katie Leftridge,Cha Cha Fisher,Emelie Frederikson,Stevenson,Principia,781,20170318
781,2,Grace Coleman,Trang Christensen,Nikki Matters,Devon Marunde,Stevenson,Principia,781,20170318
781,3,Sara Bollinger,Samantha Perillo,Noelle Shoemake,Sophia Hathaway,Principia,Stevenson,781,20170318
781,4,Darian Hileman,Megan Hren,Makena Patterson,Victoria Vandiver,Stevenson,Principia,781,20170318
781,5,Deanna Bradbury,Chelsea Ireland,Amanda Roberts,Mesa Goebel,Stevenson,Principia,781,20170318


In [408]:
keys = s[0].keys()
with open('matches_dict.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(s)

In [127]:
df_matchups = df_matchups.drop_duplicates()
df_matchups

,Position,TeamAPlayer1,TeamAPlayer2,TeamBPlayer1,TeamBPlayer2,Winner,Loser,Compid,Event Date
780,1,Alexandra Spadaro,Nikki McDaniel,Kristen Brooks,Samantha Perillo,Webber,Stevenson,780,20170316
780,2,Carlee Baxter,Tessa Colden,Lauren Weaver,Katie Leftridge,Webber,Stevenson,780,20170316
780,3,Brianna Baez,Michelle Calderon,Trang Christensen,Grace Coleman,Webber,Stevenson,780,20170316
780,4,Amber Lake,Hannah Tannone,Darian Hileman,Megan Hren,Webber,Stevenson,780,20170316
780,5,Katie Schueman,Laura Embry,Deanna Bradbury,Chelsea Ireland,Webber,Stevenson,780,20170316
781,1,Lauren Weaver,Katie Leftridge,Cha Cha Fisher,Emelie Frederikson,Stevenson,Principia,781,20170318
781,2,Grace Coleman,Trang Christensen,Nikki Matters,Devon Marunde,Stevenson,Principia,781,20170318
781,3,Sara Bollinger,Samantha Perillo,Noelle Shoemake,Sophia Hathaway,Principia,Stevenson,781,20170318
781,4,Darian Hileman,Megan Hren,Makena Patterson,Victoria Vandiver,Stevenson,Principia,781,20170318
781,5,Deanna Bradbury,Chelsea Ireland,Amanda Roberts,Mesa Goebel,Stevenson,Principia,781,20170318


# Ratings
## Ratings > Set Defaults
### Ratings > Set Defaults > Player Ratings

In [123]:
def set_default_player_rating(df):
    df_temp = df.copy()
    df_temp['mu'] = Rating().mu
    df_temp['sigma'] = Rating().sigma
    
    df_temp = df_temp.reset_index()
    df_temp = df_temp.iloc[:, 1:]
    return df_temp

df_player_rating = set_default_player_rating(df_players)
#df_player_rating[df_player_rating['Name'] == 'Mandi Orellana']
df_player_rating

,Name,Year,Height,Hometown/HS,Team,mu,sigma
0,Alyssa Slagerman,Fr,5'7,"Brentwood, Ca",UCLA,25,8.333333
1,Cami Sanchez,Fr,5'10,"Fountain Valley, Ca",UCLA,25,8.333333
2,Chanti Holroyd,So,5-9,"Mountain View, California",UCLA,25,8.333333
3,Elise Zappia,Sr,5'8,"Huntington Beach, California",UCLA,25,8.333333
4,Hannah Phair,Fr,5'10,"Huntington Beach, Ca",UCLA,25,8.333333
5,Izzy Carey,Jr,5-11,"Westlake Village, California",UCLA,25,8.333333
6,Jamie McQuarrie,Fr,6'0,"Encinitas, Ca",UCLA,25,8.333333
7,Lexi Jasper-Baylin,Sr,5'6,"El Cajon, Ca",UCLA,25,8.333333
8,Lily Justine,So,5-10,"Chico, California",UCLA,25,8.333333
9,Mac May,Fr,6'3,"Dubuque, IA",UCLA,25,8.333333


### Ratings > Lookup > Player Ratings

In [131]:
def lookup_player_rating(name, teams, df): 
    name = name.strip()
    teams = [a.strip() for a in teams]
    player_info =  df[(df['Name'] == name) & (df['Team'].isin(teams))].max()
    #player_info['pid'] = player_info.index
    return player_info

lookup_player_rating('Mandi Orellana',  ['South Carolina', 'LSU'], df_player_rating)

#df_player_rating[df_player_rating['Name'] == 'Mandi Orellana']['mu'].item()

Name                      Mandi Orellana
Year                                  Sr
Height                               5'7
Hometown/HS    Guatemala City, Guatemala
Team                                 LSU
mu                               21.8917
sigma                            7.77436
dtype: object

## Ratings > Update
### Ratings > Update > Update Player Ratings

In [181]:
def update_player_ratings(matches, player_rating, players):
    for idx, rows in matches.iterrows():
        p1_lookup =  lookup_player_rating(rows['TeamAPlayer1'],[rows['Winner'], rows['Loser']], player_rating )
        r1 = Rating(mu=p1_lookup.mu, sigma=p1_lookup.sigma)
        
        p2_lookup =  lookup_player_rating(rows['TeamAPlayer2'],[rows['Winner'], rows['Loser']], player_rating )
        r2 = Rating(mu=p2_lookup.mu, sigma=p2_lookup.sigma)
        
        p3_lookup =  lookup_player_rating(rows['TeamBPlayer1'],[rows['Winner'], rows['Loser']], player_rating )
        r3 = Rating(mu=p3_lookup.mu, sigma=p3_lookup.sigma)

        p4_lookup =  lookup_player_rating(rows['TeamBPlayer2'],[rows['Winner'], rows['Loser']], player_rating )
        r4 = Rating(mu=p4_lookup.mu, sigma=p4_lookup.sigma)
                
        
        t1 = [r1,r2]
        t2 = [r3,r4]

        if p1_lookup['Team'] == rows['Winner'].strip():
            my_ranks = [0,1]
        else:
            my_ranks = [1,0]
        
        (r1_new, r2_new), (r3_new, r4_new) = rate([t1, t2], ranks=my_ranks)

        #print rows.iloc[0],  "winner: ", rows.iloc[5],  "loser: ", rows.iloc[6]
        
        my_index1 = player_rating[player_rating['Name'] == rows.iloc[1]].index
        #print my_index1, 
        player_rating.loc[my_index1, 'mu'] = r1_new.mu
        player_rating.loc[my_index1, 'sigma'] = r1_new.sigma
        #print rows.iloc[1] , "(before:", r1.mu, ", after:", r1_new.mu, ")"
        
        my_index2 = player_rating[player_rating['Name'] == rows.iloc[2]].index
        #print my_index2,
        player_rating.loc[my_index2, 'mu'] = r2_new.mu
        player_rating.loc[my_index2, 'sigma'] = r2_new.sigma
        #print rows.iloc[2] , "(before:", r2.mu, ", after:", r2_new.mu, ")" 
        
        my_index3 = player_rating[player_rating['Name'] == rows.iloc[3]].index
        #print my_index3,
        player_rating.loc[my_index3, 'mu'] = r3_new.mu
        player_rating.loc[my_index3, 'sigma'] = r3_new.sigma
        #print rows.iloc[3] , "(before:", r3.mu, ", after:", r3_new.mu, ")" 

        my_index4 = player_rating[player_rating['Name'] == rows.iloc[4]].index
        #print my_index4,
        player_rating.loc[my_index4, 'mu'] = r4_new.mu    
        player_rating.loc[my_index4, 'sigma'] = r4_new.sigma    
        #print rows.iloc[4] , "(before:", r4.mu, ", after:", r4_new.mu, ")" 

         
        #print   "\n"
    return player_rating

#df_player_rating = set_default_player_rating(df_players)
#df_2018_matchups = select_matchups( 20180000, 20180900).sort_values(['Event Date', 'Compid', 'Position'], ascending=True).drop_duplicates()
#df_2018_matchups

#update_player_ratings(df_2018_matchups, df_player_rating, df_players) 

In [136]:
df_player_rating = set_default_player_rating(df_players)
df_player_rating

,Name,Year,Height,Hometown/HS,Team,mu,sigma
0,Alyssa Slagerman,Fr,5'7,"Brentwood, Ca",UCLA,25,8.333333
1,Cami Sanchez,Fr,5'10,"Fountain Valley, Ca",UCLA,25,8.333333
2,Chanti Holroyd,So,5-9,"Mountain View, California",UCLA,25,8.333333
3,Elise Zappia,Sr,5'8,"Huntington Beach, California",UCLA,25,8.333333
4,Hannah Phair,Fr,5'10,"Huntington Beach, Ca",UCLA,25,8.333333
5,Izzy Carey,Jr,5-11,"Westlake Village, California",UCLA,25,8.333333
6,Jamie McQuarrie,Fr,6'0,"Encinitas, Ca",UCLA,25,8.333333
7,Lexi Jasper-Baylin,Sr,5'6,"El Cajon, Ca",UCLA,25,8.333333
8,Lily Justine,So,5-10,"Chico, California",UCLA,25,8.333333
9,Mac May,Fr,6'3,"Dubuque, IA",UCLA,25,8.333333


In [144]:
#df_2018_matchups = select_matchups( 20180000, 20180900).sort_values(['Event Date', 'Compid', 'Position'], ascending=True).drop_duplicates()
df_2018_matchups = df_matchups[df_matchups['Event Date'].astype(int) > 20180000].reset_index()
df_2018_matchups = df_2018_matchups.iloc[:,1:]
df_2018_matchups

,Position,TeamAPlayer1,TeamAPlayer2,TeamBPlayer1,TeamBPlayer2,Winner,Loser,Compid,Event Date
0,1,Avery Keathley,Kersti Whitney,Kayla Tinker,Gabby Morrison,Concordia,Vanguard,790,20180215
1,2,Vanessa Bernzen,Tristan Self,Kelli Dowell,Hannah Holder,Concordia,Vanguard,790,20180215
2,3,Marcela Frazzoni,Kyra Auten,Dani Erdelatz,Charity Dennis,Concordia,Vanguard,790,20180215
3,4,Asia O'Neil,Jessica Diederich,Karly Cantu,Eliana Johnson,Concordia,Vanguard,790,20180215
4,5,Riley Gill,Kekela Hendrickson,Karli Burk,Bree Bolton,Concordia,Vanguard,790,20180215
5,1,Jewel Yandall,Sophia Loiola,Kacey Mathieson,Jadelyn Forgiarini,Orange Coast College,El Camino College,791,20180216
6,2,Sarah Dyer,Jaylin Motley,Sierra Lim,Hannah Del Barto,Orange Coast College,El Camino College,791,20180216
7,3,Aiko Waters,Phina Leilua,Rylee Adair,Devyn Holland,El Camino College,Orange Coast College,791,20180216
8,4,Glorious Owens,Jossie Tomita,Hannah Reid,Molly Thee,El Camino College,Orange Coast College,791,20180216
9,5,Kiana Takahashi,Melissa Euyoque,Enjelica Glatti,Georgia Grisham,El Camino College,Orange Coast College,791,20180216


# D1 Matchups

In [182]:
d1_colleges = list(df_teams[df_teams['Division'] == 'DI'].College.unique())

df_2018_d1_matchups = df_2018_matchups[df_2018_matchups['Winner'].isin(d1_colleges)].reset_index()
df_2018_d1_matchups = df_2018_d1_matchups.iloc[:,1:]
df_2018_d1_matchups

,Position,TeamAPlayer1,TeamAPlayer2,TeamBPlayer1,TeamBPlayer2,Winner,Loser,Compid,Event Date
0,1,Carly Perales,Sunniva Helland-Hansen,Mackenzie Redner,Savanna Cary,Stetson,Eckerd,799,20180223
1,2,Darby Dunn,Sammee Thomas,Paula Hoffman,Kaitlyn Sibson,Stetson,Eckerd,799,20180223
2,3,Rebecca Ingram,Quinci Birker,Madison Whiting,Arielle Erillo,Stetson,Eckerd,799,20180223
3,4,Rachel Noble,Samantha Harris,Rachel Cornatzer,Stephanie Bogda,Stetson,Eckerd,799,20180223
4,5,Alexa Richardson,Katinka Krahn,Kali Boatwright,Jordan Duffy,Stetson,Eckerd,799,20180223
5,1,Shannon Williams,Ali Denney,Tiffany Creamer,Brooke Weiner,South Carolina,Georgia State,800,20180223
6,2,Lydia Dimke,Katie Smith,Ashley McGinn,Allie Elson,South Carolina,Georgia State,800,20180223
7,3,Julia Mannisto,Carly Schneider,Georgia Johnson,Eden Hawes,South Carolina,Georgia State,800,20180223
8,4,Leah Perri,Macie Tendrich,Kate Novack,Rebekah Brodbeck,South Carolina,Georgia State,800,20180223
9,5,Franky Harrison,Hannah Edelman,Amie Held,Maddie Gordon,South Carolina,Georgia State,800,20180223


## D1 Matchups > All Players

In [183]:
df_player_rating = set_default_player_rating(df_players)
update_player_ratings(df_2018_d1_matchups, df_player_rating, df_players)
df_player_rating.sort_values('mu', ascending=False)
df_player_rating.sort_values('mu', ascending=False).head(22)

,Name,Year,Height,Hometown/HS,Team,mu,sigma
8,Lily Justine,So,5-10,"Chico, California",UCLA,43.681516,2.824418
43,Haley Hallgren,Fr,6'1,"Southlake, Tx",USC,43.329641,2.238813
53,Tina Graudina,Fr,6'0,"Jurmala, Latvia",USC,41.966408,2.922260
38,Victoria Paranagua,Sr,5'11,"Austin, Tx",Florida State,40.505120,2.484174
11,Megan McNamara,Jr,5-9,"Vancouver, BC, Canada",UCLA,40.175518,2.441323
13,Nicole McNamara,Jr,5-9,"Vancouver, BC, Canada",UCLA,40.175518,2.441323
29,Katie Horton,R-Sr,6'1,"Westland, Mich",Florida State,39.214020,3.254060
28,Hailey Luke,R-Sr,6'0,"Altamonte Springs, Fl",Florida State,39.195730,3.259326
17,Zana Muno,Jr,5'10,"Hermosa Beach, Ca",UCLA,39.093921,3.160932
448,Taylor Nelson,Sr,6'0,"Granite Bay, Ca",Cal Poly SLO,38.981464,3.414124


## D1 Matchups > Position 1

In [184]:
df_player_rating_p1 = set_default_player_rating(df_players)
df_2018_d1p1_matchups = df_2018_d1_matchups[df_2018_d1_matchups['Position'].astype(int) == 1]
update_player_ratings(df_2018_d1p1_matchups, df_player_rating_p1, df_players)
df_player_rating_p1.sort_values('mu', ascending=False).head(22)

,Name,Year,Height,Hometown/HS,Team,mu,sigma
53,Tina Graudina,Fr,6'0,"Jurmala, Latvia",USC,41.385491,3.127093
11,Megan McNamara,Jr,5-9,"Vancouver, BC, Canada",UCLA,39.912827,2.530033
13,Nicole McNamara,Jr,5-9,"Vancouver, BC, Canada",UCLA,39.912827,2.530033
39,Abril Bustamante,Jr,6'0,"Redondo Beach, Ca",USC,37.631245,2.891385
38,Victoria Paranagua,Sr,5'11,"Austin, Tx",Florida State,37.601392,2.841934
66,Ka'iwi Schucht,Sr,5'9,"Escondido, Ca",Hawaii,36.525595,4.045355
62,Emily Maglio,Jr,6'3,"Coquitlam, B.C., Can",Hawaii,36.525595,4.045355
123,Nele Barber,Sr,6'0,"Berlin, Germany",Long Beach State,35.657780,3.119632
173,Claire Coppola,So,6'1,"Scottsdale, Az",LSU,35.450965,2.636264
181,Kristen Nuss,So,5'6,"New Orleans, La",LSU,35.450965,2.636264


## D1 Matchups > Position 2

In [185]:
df_player_rating_p2 = set_default_player_rating(df_players)
df_2018_d1p2_matchups = df_2018_d1_matchups[df_2018_d1_matchups['Position'].astype(int) == 2]
update_player_ratings(df_2018_d1p2_matchups, df_player_rating_p2, df_players)
df_player_rating_p2.sort_values('mu', ascending=False).head(22)

,Name,Year,Height,Hometown/HS,Team,mu,sigma
8,Lily Justine,So,5-10,"Chico, California",UCLA,44.131869,2.909919
29,Katie Horton,R-Sr,6'1,"Westland, Mich",Florida State,42.441873,4.429736
28,Hailey Luke,R-Sr,6'0,"Altamonte Springs, Fl",Florida State,42.411765,4.442434
345,Madalyn Roh,Jr,5'11,"Scottsdale, Az",Pepperdine,38.878084,4.074466
337,Brook Bauer,Fr,6'1,"Fort Lauderdale, Fl",Pepperdine,38.878084,4.074466
338,Corinne Quiggle,Sr,5'10,"Ponte Vedra Beach, Fl",Pepperdine,38.220430,4.740924
339,Deahna Kraft,So,6'0,"Seattle, Wa",Pepperdine,38.220430,4.740924
193,Ali Denney,So,5'6,"Jupiter,Fl",South Carolina,38.143611,5.470877
213,Shannon Williams,Jr,5'10,"Ashburn, Va",South Carolina,38.143611,5.470877
68,Lea Monkhouse,Fr,6'0,"Mississauga, Canada",Hawaii,37.524897,4.425926


## D1 Matchups > Position 3

In [186]:
df_player_rating_p3 = set_default_player_rating(df_players)
df_2018_d1p3_matchups = df_2018_d1_matchups[df_2018_d1_matchups['Position'].astype(int) == 3]
update_player_ratings(df_2018_d1p3_matchups, df_player_rating_p3, df_players)
df_player_rating_p3.sort_values('mu', ascending=False).head(22)

,Name,Year,Height,Hometown/HS,Team,mu,sigma
51,Sammy Slater,Fr,5'10,"Ventura, Ca",USC,43.519271,5.187601
60,Carly Kan,Sr,5'9,"Honolulu, Hi",Hawaii,42.081224,3.774082
67,Laurel Weaver,Sr,5'11,"Winnetka, Ca",Hawaii,40.709018,3.838889
36,Sara Putt,Jr,5'11,"Jupiter, Fl",Florida State,40.215833,3.764765
31,Macy Jerger,Jr,6'1,"Fort Myers, Fl",Florida State,39.894018,4.191090
43,Haley Hallgren,Fr,6'1,"Southlake, Tx",USC,39.211912,3.378407
16,Savvy Simo,So,5'10,"Rancho Santa Fe, Ca",UCLA,38.069072,3.105103
52,Terese Cannon,Sr,6'3,"Pittsford, N.Y.",USC,37.808563,5.642778
17,Zana Muno,Jr,5'10,"Hermosa Beach, Ca",UCLA,37.440881,3.655014
432,Adlee Van Winden,Jr,6'1,"Napa, Ca",Cal Poly SLO,35.973484,3.983916


## D1 Matchups > Position 4

In [187]:
df_player_rating_p4 = set_default_player_rating(df_players)
df_2018_d1p4_matchups = df_2018_d1_matchups[df_2018_d1_matchups['Position'].astype(int) == 4]
update_player_ratings(df_2018_d1p4_matchups, df_player_rating_p4, df_players)
df_player_rating_p4.sort_values('mu', ascending=False).head(22)

,Name,Year,Height,Hometown/HS,Team,mu,sigma
3,Elise Zappia,Sr,5'8,"Huntington Beach, California",UCLA,40.326741,3.364683
381,Emma Doud,So,5'9,"Alameda, Ca",LMU,38.627643,5.886282
57,Ari Homayun,Jr,5'8,"Woodland Hills, Ca",Hawaii,38.274934,2.782579
55,Amy Ozee,So,6'1,"Haiku, Hi",Hawaii,38.274934,2.782579
220,Jonny Baham,Fr,5'6,"Carlsbad, Ca",Arizona,37.864724,4.140031
24,Brooke Kuhlman,Jr,5'10,"Naples, Fl",Florida State,37.162468,3.817856
349,Skylar Caputo,Jr,5'10,"Manhattan Beach, Ca",Pepperdine,36.854755,3.539618
335,Alexis Filippone,Fr,6'2,"Del Mar, Ca",Pepperdine,36.854755,3.539618
34,Molly McBain,So,5'9,"Toronto, Canada",Florida State,36.791643,3.814126
124,Nicci Reinking,Fr,5'10,"Fountain Valley, Ca",Long Beach State,35.088645,3.473981


## D1 Matchups > Position 5

In [188]:
df_player_rating_p5 = set_default_player_rating(df_players)
df_2018_d1p5_matchups = df_2018_d1_matchups[df_2018_d1_matchups['Position'].astype(int) == 5]
update_player_ratings(df_2018_d1p5_matchups, df_player_rating_p5, df_players)
df_player_rating_p5.sort_values('mu', ascending=False).head(22)

,Name,Year,Height,Hometown/HS,Team,mu,sigma
43,Haley Hallgren,Fr,6'1,"Southlake, Tx",USC,48.660928,4.042437
26,Eva Torruella,R-So,6'0,"San Juan, Puerto Rico",Florida State,41.452872,4.027659
27,Francesca Goncalves,So,5'10,"Dallas, Tx",Florida State,40.961175,3.616781
146,Maddie Micheletti,Fr,5'9,"Manhattan Beach, Ca",Cal,37.522260,3.777464
63,Hannah Zalopany,Sr,5'9,"Spokane, Wa",Hawaii,37.367179,2.896499
478,Erica Brok,Fr,5'9,"Ormond Beach, Fl",FAU,36.709530,5.951729
486,Mackenzie Morris,Fr,5'10,"Jacksonville, Fl",FAU,36.709530,5.951729
183,Maddie Ligon,Jr,5'8,"Bedford, Tx",LSU,36.655751,3.408135
70,Paige Dreeuws,Fr,5'7,"San Marcos, Ca",Hawaii,36.379313,3.015508
12,Megan Muret,Fr,5'9,"Encinitas, Ca",UCLA,36.099159,3.282881


## Players > Rating

In [504]:
print df_matchups[df_matchups['TeamBPlayer1'] == 'Tina Graudina']
print df_matchups[df_matchups['TeamAPlayer1'] == 'Tina Graudina']

     Position     TeamAPlayer1        TeamAPlayer2   TeamBPlayer1  \
930         1      Brook Bauer         Madalyn Roh  Tina Graudina   
1150        1        Carly Kan       Laurel Weaver  Tina Graudina   
1194        1     Emily Maglio      Ka'iwi Schucht  Tina Graudina   
1589        1  Corinne Quiggle        Deahna Kraft  Tina Graudina   
1838        1  Nicole McNamara      Megan McNamara  Tina Graudina   
1845        1  Nicole McNamara      Megan McNamara  Tina Graudina   
1846        1  Nicole McNamara      Megan McNamara  Tina Graudina   
1860        1   Vanessa Freire  Victoria Paranagua  Tina Graudina   

          TeamBPlayer2  Winner          Loser  Compid Event Date  
930   Abril Bustamante     USC     Pepperdine     930   20180303  
1150  Abril Bustamante     USC         Hawaii    1150   20180317  
1194  Abril Bustamante  Hawaii            USC    1194   20180318  
1589  Abril Bustamante     USC     Pepperdine    1589   20180410  
1838  Abril Bustamante     USC           UC

# Probability and Statistics

In [189]:
def win_probability(team1, team2):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts = global_env()
    return ts.cdf(delta_mu / denom)

In [232]:
def actual_match(compid, df):
    return df[df['Compid'].astype(int) == compid].groupby('Winner')['Position'].count()

actual_match(1863, df_matchups)

Winner
FIU    2
USC    3
Name: Position, dtype: int64

In [462]:
def predict_match(compid, df):
    match = df[df['Compid'].astype(int) == compid].copy().reset_index().iloc[:,1:]
    for idx, rows in match.iterrows():
        r = {}
        p_lookup = {}
        t = {}
        for i in range(1,5):
            p_lookup[i] = lookup_player_rating(rows.iloc[i],[rows.iloc[5], rows.iloc[6]], df_player_rating )
            r[i] = Rating(mu=p_lookup[i].mu, sigma=p_lookup[i].sigma)
            #match[i] = r[i]
    
        t[1] = [r[1], r[2]]
        t[2] = [r[3], r[4]]
        
        win_prob = win_probability(t[1], t[2])
    
        
        match.loc[idx, 'Probability'] = win_prob 
        #print win_prob
        if (win_prob >.60):
            match.loc[idx, 'Predicted Winner'] = p_lookup[1]['Team'] 
            match.loc[idx, 'Distance From Draw'] = (win_prob - .60)
            match.loc[idx, 'T1 Win'] = 1
        elif (win_prob <.40):
            match.loc[idx, 'Predicted Winner'] = p_lookup[3]['Team'] 
            match.loc[idx, 'Distance From Draw'] = (.40-win_prob )
            match.loc[idx, 'T1 Win'] = 0
        else:
            match.loc[idx, 'Predicted Winner'] = 'Draw'
            match.loc[idx, 'Distance From Draw'] = 0
            match.loc[idx, 'T1 Win'] = .5
        
    
    return match

_df = {}
temp = None
for a in range(780, 1870):
    print a,
    _df = predict_match(a, df_matchups)
    _df = pd.concat([_df, temp], axis=0)
    temp = _df


total_no_draw = len(_df[_df['Winner'] != 'Draw'])
accurate = len(_df[_df['Winner'] == _df['Predicted Winner']])
print accurate, total_no_draw, accurate*100/total_no_draw

_df

780 781 782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866 867 868 869 870 871 872 873 874 875 876 877 878 879 880 881 882 883 884 885 886 887 888 889 890 891 892 893 894 895 896 897 898 899 900 901 902 903 904 905 906 907 908 909 910 911 912 913 914 915 916 917 918 919 920 921 922 923 924 925 926 927 928 929 930 931 932 933 934 935 936 937 938 939 940 941 942 943 944 945 946 947 948 949 950 951 952 953 954 955 956 957 958 959 960 961 962 963 964 965 966 967 968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020 1021 1022 1023 

,Compid,Distance From Draw,Event Date,Loser,Position,Predicted Winner,Probability,T1 Win,TeamAPlayer1,TeamAPlayer2,TeamBPlayer1,TeamBPlayer2,Winner
0,1869,0.000000,20180505,Hawaii,1,Draw,0.401969,0.5,Victoria Paranagua,Vanessa Freire,Emily Maglio,Ka'iwi Schucht,Florida State
1,1869,0.101197,20180505,Hawaii,2,Florida State,0.701197,1.0,Katie Horton,Hailey Luke,Lea Monkhouse,Morgan Martin,Florida State
2,1869,0.000000,20180505,Florida State,3,Draw,0.466069,0.5,Macy Jerger,Sara Putt,Carly Kan,Laurel Weaver,Hawaii
3,1869,0.080847,20180505,Florida State,4,Hawaii,0.319153,0.0,Brooke Kuhlman,Molly McBain,Amy Ozee,Ari Homayun,Hawaii
4,1869,0.000000,20180505,Hawaii,5,Draw,0.478062,0.5,Madison Fitzpatrick,Francesca Goncalves,Paige Dreeuws,Hannah Zalopany,Florida State
0,1868,0.234434,20180505,LSU,1,UCLA,0.834434,1.0,Nicole McNamara,Megan McNamara,Claire Coppola,Kristen Nuss,UCLA
1,1868,0.299117,20180505,LSU,2,UCLA,0.899117,1.0,Sarah Sponcil,Lily Justine,Haleigh Nelson,Mandi Orellana,UCLA
2,1868,0.311550,20180505,LSU,3,UCLA,0.911550,1.0,Zana Muno,Savvy Simo,Olivia Powers,Hunter Domanski,UCLA
0,1867,0.198677,20180505,Pepperdine,1,USC,0.798677,1.0,Abril Bustamante,Tina Graudina,Corinne Quiggle,Deahna Kraft,USC
1,1867,0.045169,20180505,USC,2,Pepperdine,0.354831,0.0,Sammy Slater,Terese Cannon,Brook Bauer,Madalyn Roh,Pepperdine


In [467]:
import numpy as np
from sklearn.metrics import accuracy_score
y_true = list(_df['T1 Win'])
y_pred = list(_df['Probability'])
accuracy_score(y_true, y_pred)

#accuracy_score(y_true, y_pred, normalize=False)


ValueError: continuous is not supported

In [494]:
_df = _df.reset_index()
_df = _df.iloc[:,1:]
_df['Actual'] = 0
idx = _df.index[_df['Winner'] == _df['Predicted Winner']]
#_df.index = idx, 'Actual'] = 1
#df.Actual.where[_df['Winner'] == _df['Predicted Winner']])

_df['Actual'] = np.where(_df['Winner'] == _df['Predicted Winner'], 1, 0)
_df

,Compid,Distance From Draw,Event Date,Loser,Position,Predicted Winner,Probability,T1 Win,TeamAPlayer1,TeamAPlayer2,TeamBPlayer1,TeamBPlayer2,Winner,Actual
0,1869,0.000000,20180505,Hawaii,1,Draw,0.401969,0.5,Victoria Paranagua,Vanessa Freire,Emily Maglio,Ka'iwi Schucht,Florida State,0
1,1869,0.101197,20180505,Hawaii,2,Florida State,0.701197,1.0,Katie Horton,Hailey Luke,Lea Monkhouse,Morgan Martin,Florida State,1
2,1869,0.000000,20180505,Florida State,3,Draw,0.466069,0.5,Macy Jerger,Sara Putt,Carly Kan,Laurel Weaver,Hawaii,0
3,1869,0.080847,20180505,Florida State,4,Hawaii,0.319153,0.0,Brooke Kuhlman,Molly McBain,Amy Ozee,Ari Homayun,Hawaii,1
4,1869,0.000000,20180505,Hawaii,5,Draw,0.478062,0.5,Madison Fitzpatrick,Francesca Goncalves,Paige Dreeuws,Hannah Zalopany,Florida State,0
5,1868,0.234434,20180505,LSU,1,UCLA,0.834434,1.0,Nicole McNamara,Megan McNamara,Claire Coppola,Kristen Nuss,UCLA,1
6,1868,0.299117,20180505,LSU,2,UCLA,0.899117,1.0,Sarah Sponcil,Lily Justine,Haleigh Nelson,Mandi Orellana,UCLA,1
7,1868,0.311550,20180505,LSU,3,UCLA,0.911550,1.0,Zana Muno,Savvy Simo,Olivia Powers,Hunter Domanski,UCLA,1
8,1867,0.198677,20180505,Pepperdine,1,USC,0.798677,1.0,Abril Bustamante,Tina Graudina,Corinne Quiggle,Deahna Kraft,USC,1
9,1867,0.045169,20180505,USC,2,Pepperdine,0.354831,0.0,Sammy Slater,Terese Cannon,Brook Bauer,Madalyn Roh,Pepperdine,1


In [452]:
new = _df[['Predicted Winner', 'Winner', 'Loser', 'Probability', 'Distance From Draw']].sort_values('Distance From Draw', ascending=False)
for idx, rows in new.iterrows():
    print rows[0],rows[1],  rows[4]

UCLA UCLA 0.449995801024
USC USC 0.44996500132
Florida State Florida State 0.449918758715
UCLA UCLA 0.449903729118
Long Beach State Long Beach State 0.449872676374
Pepperdine Pepperdine 0.449848128189
UCLA UCLA 0.44984346638
Cal Poly SLO Cal Poly SLO 0.449840650007
Cal Cal 0.449805958334
UCLA UCLA 0.449792866828
USC USC 0.449781972953
USC USC 0.449781972953
Stetson Stetson 0.449777900169
Stetson Stetson 0.449777900169
Hawaii Hawaii 0.449772880974
Pepperdine Pepperdine 0.44975450315
UCLA UCLA 0.44971796524
Hawaii Hawaii 0.449679452693
Hawaii Hawaii 0.449677688158
LSU LSU 0.449594763002
Stetson Stetson 0.449589186606
Cal Poly SLO Cal Poly SLO 0.449581912768
UCLA UCLA 0.449555373676
UCLA UCLA 0.44955225683
Long Beach State Long Beach State 0.449475350637
South Carolina South Carolina 0.449428055775
Cal Cal 0.449409011334
Stetson Stetson 0.44937999297
South Carolina South Carolina 0.449370743513
USC USC 0.449357087668
Stetson Stetson 0.449346126793
Saint Mary's Saint Mary's 0.449330014176


## 2019 Potential (Look for Pre-season Rankings)
Takes a look at team without last year's seniors

In [500]:
df_2019_Potential = df_player_rating[~df_player_rating['Year'].isin(['Sr','R-Sr', 'GS'])].sort_values('mu', ascending=False)
df_2019_Potential = df_2019_Potential.groupby('Team')['mu'].nlargest(8).sum(level=0).sort_values(ascending=False).to_frame()
df_2019_Potential.columns = ['Total K Score']
df_2019_Potential
#filter out D1s

,Total K Score
Team,
UCLA,299.066099
Pepperdine,285.937639
Florida State,282.579811
Hawaii,280.389841
USC,271.952718
LSU,266.957376
Long Beach State,259.337170
Stetson,255.976976
Cal Poly SLO,250.897406


In [ ]:
#Strength of schedule?

## 2018 Post Season Team Strength Ranking By Top 10 Cumulative Player Score

In [443]:
df_2018_final = df_player_rating.sort_values('mu', ascending=False)
df_2018_final = df_2018_final.groupby('Team')['mu'].nlargest(10).sum(level=0).sort_values(ascending=False).to_frame()
df_2018_final.columns = ['Total K Score']
df_2018_final

,Total K Score
Team,
Florida State,370.428862
UCLA,367.920791
Hawaii,364.015556
Pepperdine,351.316729
USC,341.781269
Long Beach State,331.825753
LSU,329.397887
Cal Poly SLO,325.496092
Stetson,313.400536


In [ ]:
def predictMatch(_compid, _df):
    _match = df_all_matchups[df_all_matchups['CompID'] == _compid].iloc[:,0:5]
    wins = {}

    for i in range(0,len(_match)):
        r = [Rating(lookupPlayerRating(_match.iloc[i,j], predict_ranking))  for j in range(1,5)]

        t1 = [r[0], r[1]]
        t2 = [r[2], r[3]]

        win_prob = win_probability(t1, t2)

        #print [(_match.iloc[i,j]+"("+lookupPlayer(_match.iloc[i,j], _df)['Team']+")") for j in range(1,5) ],

        if win_prob > .50:
            val = max(lookupPlayer(_match.iloc[i, 1], _df)['Team'], lookupPlayer(_match.iloc[i, 2], _df)['Team'])
            #print val, ' favored'
            try:
                wins[val] += 1
            except:
                wins[val] =  1
        elif win_prob < .50:
            val = max(lookupPlayer(_match.iloc[i, 3], _df)['Team'], lookupPlayer(_match.iloc[i, 4], _df)['Team'])
            #print val, ' favored'
            try:
                wins[val] += 1
            except:
                wins[val] =  1
        else:
            val = 'Draw'
            try:
                wins[val] += 1
            except:
                wins[val] =  1
                
    match_winner = max(wins.iteritems(), key=operator.itemgetter(1))[0]      
    
    return wins, match_winner

print "Predicted:" + str(predictMatch(1816, df_player_ratings))
print "Actual:" + str(actualMatch(1816, df_player_ratings))

# Pairs Rating
I shall assert that a pairs rating is more than the sum of ratings of players. In additional to 

In [ ]:
df_2018_matchups
t1 = df_2018_matchups[['TeamAPlayer1', 'TeamAPlayer2']].copy()
t2 = df_2018_matchups[['TeamBPlayer1', 'TeamBPlayer2']].copy()
t1.columns = ['P1', 'P2']
t2.columns = ['P1', 'P2']
df_pairs = pd.concat([t1,t2], axis=0)
df_pairs = pd.merge(df_pairs, df_player_ratings[['Player','Team']], how='inner', left_on='P1', right_on='Player')[['P1', 'P2', 'Team']]
df_pairs.drop_duplicates(inplace=True)
df_pairs

# Team Rating
N:M unbalanced match – [(r1,), (r2, r3, r4)]

# Ranking

In [ ]:
def get_positional_ranking(startdate, enddate, position, division ='DI' ):
    df_pr = setDefaultPlayerRating(df_players)
    new_df = select_matchups( 20180000, 20180900)

    df_player_ratings_new = updatePlayerRatings(_new_df, _df_player_rating)
    
    top_players = df_player_ratings_new.sort_values('mu', ascending=False).reset_index()
    top_players = top_players.iloc[:,1:]
    top_players['Rank'] = top_players.index+1
    return top_players

#pos1_ranking = getPositionalRanking('DI', 20180000, 20190000, 1, df_all_players)
#pos2_ranking = getPositionalRanking('DI', 20180000, 20190000, 2, df_all_players)
current_ranking = getPositionalRanking('DI', 2018000, 20180600, 'all', df_all_players)
current_ranking

In [ ]:
current_ranking['Year'].unique()
#current_ranking[current_ranking['Year'] == None]
curr = current_ranking[current_ranking['Year'] != '']
curr = curr[~curr['Year'].isnull() ]
curr = curr[~curr['Year'].str.contains('Sr')]
curr.iloc[:60,:].groupby(['Team'])['mu'].sum().sort_values(ascending=False).to_frame()

# Pre-season rankings

In [ ]:
current_ranking['Year'].unique()
current_ranking[current_ranking['Year'] == None]
curr = current_ranking[current_ranking['Year'] != '']
curr = curr[~curr['Year'].isnull() ]
#curr = curr[~curr['Year'].str.contains('Sr')]
curr.iloc[:60,:].groupby(['Team'])['mu'].sum().sort_values(ascending=False).to_frame()


In [ ]:
def lookup_player_rating(name, df):
    v_idx = max(_df.index[_df.Player.str.contains(_name.lstrip())])
    v_idx = max(_df.index[_df.Player.str.contains(_name.rstrip())])
    v = _df.loc[v_idx]
    return v['mu'], v['sigma']

In [ ]:
for i in range(1816,1817):
    print  predictMatch(i, df_player_ratings)[1], 'Predict',
    print   actualMatch(i, df_player_ratings)[1], 'Actual',
    try:
        if predictMatch(i, df_player_ratings)[1] == actualMatch(i, df_player_ratings)[1]:
            print i, "Correct"
        else:
            print i, "Wrong"
    except:
        print i, "No Data"

In [ ]:
df_all_matchups[df_all_matchups['CompID'] == 1802]

In [ ]:

def lookupPlayer(_name, _df):
    #print _name
    v_idx = _df.index[_df.Player==_name]
    v = _df.iloc[v_idx]
    return v_idx #_df

lookupPlayer(_name, df_all_players)
df_all_players[df_all_players['Team'] == 'Sierra College']  #.unique()

# Utility

In [ ]:
def loadTable(_url, _i, _type, _columns):
    soup, table = scrapeUrl(_url, _i)
    my_labels = table[0]
    my_results = table[1:]
    
    if (_type == 'Team') & (my_labels[2] != ''):
        #for loading Player tables
        try:
            df = pd.DataFrame.from_records(my_results, columns=_columns[:4])
            df[_type] = getHeader(soup, 'College Players for ', 'player')
            df[_type + ' ID'] = _i
        except:
            try:
                df = pd.DataFrame.from_records(my_results, columns=my_labels)
                df[_type] = getHeader(soup, 'College Players for ', 'player')
                df[_type + ' ID'] = _i
            except:
                df = None
    
    
    if ((_type == 'Matchup') & (my_labels[1] != '')) > 0 :
        my_labels.insert(2, my_labels[2])
        my_labels.insert(1, my_labels[1])
        #print my_labels
        df = pd.DataFrame.from_records(my_results, columns=my_labels) 
        df[_type] = getHeader(soup, 'Competition on ', 'event')
        df[_type + ' ID'] = _i

        try:
            df['Location'] = getTag(soup, 'Location: ', 'p')
        except:
            df['Location'] = None
            
        try:
            #df['Winner'] = df['Winner'].apply(lambda x: re.sub(r'[\(\)]', '', x))
            df['Winner'] = df['Winner'].str.replace('\(', '')
            df['Winner'] = df['Winner'].str.replace('\)', '')
        except:
            df['Winner'] = None
            
        
            
        df = df[~df.iloc[:,1].str.contains("Game")]
        df = df[~df.iloc[:,3].str.contains("forfeit")]
        df = df[~df.iloc[:,4].str.contains("0-0")]
    else:
        df = None
        
    return df


def mergeResults(_url, _start, _end, _type, _columns):
    d = {i:loadTable(_url, i, _type, _columns) for i in range(_start, _end)}
    for k in d.keys():
        if d[k] is not None:
            d[k].columns = _columns 
    return pd.concat(d.values(), axis=0)

getMatchups(1802, 1803)

In [ ]:
### OLD VERSION

from datetime import datetime

def scrape_matchups(start=match_start_id, end=match_end_id):
    url = 'http://collegebeachvb.com/scores/competition/'

    d = {}
    
    for i in range(start, end):
        soup, table = scrape_url(url, i)
        header = soup.find('h1').text.strip().replace('Competition on ','')
        header = datetime.strptime(str(header), '%A, %B %d, %Y').strftime('%Y%m%d')
        
        if len(table[1:]) > 0:
            if len(table[1:][0]) == 7:
                my_labels = table[0]
                col_numbers = len(table[1:][0]) + 2
                my_results = table[1:]
                
                #print my_labels
                
                d[i] = pd.DataFrame.from_records(my_results)
                d[i]['Event Date'] = header

                #try:
                d[i][5] = d[i][5].str.replace('\(', '')
                d[i][5] = d[i][5].str.replace('\)', '')
                #except:
                #    d[i][5] = None

                d[i] = d[i][~d[i].iloc[:,1].str.contains("Game", "forfeit")]
                d[i] = d[i][~d[i].iloc[:,3].str.contains("forfeit")]
                d[i] = d[i][~d[i].iloc[:,4].str.contains("0-0")]
                d[i]['compid'] = i
        else:

            print table
            
    temp = pd.concat([d[k] for k in d.keys()], axis=0)
    temp.columns = ['Position', 'TeamAPlayer1', 'TeamAPlayer2', 'TeamBPlayer1', 'TeamBPlayer2', 'Winner', 'Score', 'Event Date', 'Compid']
        
    return temp

#df_matchups = scrape_matchups(1, 1900)

In [ ]:
### OLD
def scrape_win_lose(start=match_start_id, end=match_end_id):
    url = 'http://collegebeachvb.com/scores/competition/'
    d = {}
    
    for i in range(start, end):
        print i,":",
        soup, table = scrape_url(url, i)
        header = soup.find('h1').text.strip().replace('Competition on ','')
        header = datetime.strptime(str(header), '%A, %B %d, %Y').strftime('%Y%m%d')
        
        my_labels = table[0]
        my_results = [a for a in table[1:] if a[1] not in 'Game was unfinished']
        
        d[i] = pd.DataFrame.from_records(my_results)
        d[i]['Event Date'] = header
        
        
        if len(my_labels) < 5:
            d[i][5] = d[i][5].str.replace('\(', '')
            d[i][5] = d[i][5].str.replace('\)', '')
            d[i]['Winner'] = d[i][5]
            try:
                d[i]['Loser'] = d[i]['Winner'].apply(lambda x:  my_labels[2] if x in my_labels[1] else my_labels[1])
            except:
                d[i]['Loser'] = None

        d[i]['Compid'] = i

        

  
        temp = pd.concat([d[k] for k in d.keys()], axis=0)
        #temp = temp[[0, 'Event Date', 'Compid', 'Winner', 'Loser']]
        #temp.columns = ['Position', 'Event Date', 'Compid', 'Winner', 'Loser']


    return temp
df_winner_loser = scrape_win_lose(713,1900)
df_winner_loser

In [ ]:
OLD
def scrape_matchups(start=match_start_id, end=match_end_id):
    url = 'http://collegebeachvb.com/scores/competition/'
    d = {}
    temp = None
    for i in range(start, end):
        print i,":",
        soup, table = scrape_url(url, i)
        header = soup.find('h1').text.strip().replace('Competition on ','')
        header = datetime.strptime(str(header), '%A, %B %d, %Y').strftime('%Y%m%d')

        my_labels = table[0]
        my_results = [a for a in table[1:] if a[1] not in ['Game was unfinished', '', None, 'Forfeit']]

        d[i] = pd.DataFrame.from_records(my_results)
        d[i]['Event Date'] = header

        if ((len(my_results) > 0) & (len(my_labels) == 5)):
            d[i][5] = d[i][5].str.replace('\(', '')
            d[i][5] = d[i][5].str.replace('\)', '')
            d[i]['Winner'] = d[i][5]
            try:
                d[i]['Loser'] = d[i]['Winner'].apply(lambda x:  my_labels[2] if x in my_labels[1] else my_labels[1])
            except:
                d[i]['Loser'] = None

            d[i]['Compid'] = i

            temp = pd.concat([d[k] for k in d.keys()], axis=0)
            temp = temp.rename(columns ={0: 'Position', 1: 'TeamAPlayer1', 2: 'TeamAPlayer2', 3:'TeamBPlayer1',\
                                         4:'TeamBPlayer2', 5:'Winner', 6:'Score'})
        else:
            print "error:", i
            
    return temp
df_matchups = scrape_matchups(780, 1900)
df_matchups

In [ ]:
def lookup_winner_loser(compid, position, df_=df_matchups):
    print compid, position
    temp =  df_[(df_.Compid.astype(int) == int(compid)) & (df_.Position.astype(int) == int(position))]
    return temp.Winner.item(), temp.Loser.item()
#& (df_.Position == position))

lookup_winner_loser(1024, 3,  df_2018_matchups)

In [ ]:

## OLD
def update_player_ratings(matches, player_ratings, players):
    for idx, rows in matches.iterrows():
        t = {}
        r = {}
        
        for i in range(1,5):
            try:
                r[str(i)+'_mu'], r[str(i)+'_sigma']  = lookup_player_rating(rows.iloc[i], player_ratings )
                r[i] = Rating(mu=r[str(i)+'_mu'], sigma=r[str(i)+'_sigma'])
                print r
            except:
                print "" #hi", rows

            
        t[1] = [r[1], r[2]]
        t[2] = [r[3], r[4]]
        

        p1_team = lookup_player(rows.iloc[1], players)

        if p1_team.strip()  == rows['Winner'].strip():
            my_ranks = [0,1]
        else:
            my_ranks = [1,0]

        (r['new_1'], r['new_2']), (r['new_3'], r['new_4']) = rate([t[1], t[2]], ranks=my_ranks)

        #print idx
        for i in range(1,5):
            #print i,
            try:
                mu, sigma = lookup_player_rating(rows.iloc[i], player_ratings )
                my_idx =  player_ratings.index[player_ratings['Name'] == rows.iloc[i:i+1].item()]
                player_ratings.loc[my_idx, 'mu'] = r['new_'+str(i)].mu
                player_ratings.loc[my_idx, 'sigma'] = r['new_'+str(i)].sigma
            except:
                print "" #mu, sigma, rows.iloc[i:i+1]
        
        #print "\n"
    return player_ratings


df_2018_matchups = select_matchups( 20180000, 20180900).sort_values('Event Date', ascending=True)
update_player_ratings(df_2018_matchups, df_player_ratings, df_players)
df_player_rating